# Question 1(B)

In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [4]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [5]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [2]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

## English to Hindi

In [3]:
from google.colab import drive
import random
# Mount Google Drive
drive.mount('/content/drive')

# Access files in Google Drive
path = "/content/drive/My Drive/Colab Notebooks/B_eng_to_hin_input.txt"

with open(path, "r", encoding="utf-8") as file:
    en_data = file.readlines()

# # Print the contents of the file
#  data = data[0:1000]



Mounted at /content/drive


In [4]:
print(len(en_data))
for i in en_data:
  print(i)

1000
And now the festival of Eid is here.

The Prime Minister later flagged off the “Run For Unity” from Major Dhyan Chand National Stadium.

Saddened by his demise.

In this mutual and global pursuit I thank Prime Minister Abe and Japan for being a perfect partner.

He said this would greatly help in improving effectiveness of the exercise.

Government of India has launched ‘Pradhan Mantri Kisan Sampada Yojna’ to encourage infrastructure building.

“I bow to Loknayak Jayprakash Narayan on his birth anniversary.

Prime Minister, Shri Narendra Modi paid tributes to Mahatma Gandhi and Lal Bahadur Shastri on their birth anniversary.

Under our scheme, our fellow villagers will get loan from the government, grant, and subsidy.

He added that for the Union Government, decisions are about national interest and not political gains.

On 15th August, 2017, they unfurled the tricolour at Khardungla, in Ladakh.

My dear countrymen, Ganesh Utsav is drawing near.

His phenomenal contribution to the

In [5]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

en_sents = en_data

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_ans = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)


for i in hi_ans:
  print(i)
# print(hi_ans)

# flush the models to free the GPU memory
# del en_indic_tokenizer, en_indic_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

और अब ईद का त्योहार आ गया है।
प्रधानमंत्री ने बाद में मेजर ध्यानचंद राष्ट्रीय स्टेडियम से "रन फॉर यूनिटी" को हरी झंडी दिखाई।
उनके निधन से दुखी हूं।
इस पारस्परिक और वैश्विक प्रयास में मैं प्रधानमंत्री आबे और जापान को एक आदर्श भागीदार होने के लिए धन्यवाद देता हूं।
उन्होंने कहा कि इससे अभ्यास की प्रभावशीलता में सुधार करने में बहुत मदद मिलेगी।
भारत सरकार ने बुनियादी ढांचे के निर्माण को प्रोत्साहित करने के लिए 'प्रधानमंत्री किसान संपदा योजना' शुरू की है।
मैं लोकनायक जयप्रकाश नारायण को उनकी जयंती पर नमन करता हूं।
प्रधानमंत्री श्री नरेन्द्र मोदी ने महात्मा गांधी और लाल बहादुर शास्त्री को उनकी जयंती पर श्रद्धांजलि अर्पित की।
हमारी योजना के तहत हमारे साथी ग्रामीणों को सरकार से ऋण, अनुदान और सब्सिडी मिलेगी।
उन्होंने कहा कि केंद्र सरकार के लिए निर्णय राष्ट्रीय हित के बारे में होते हैं न कि राजनीतिक लाभ के बारे में।
15 अगस्त, 2017 को उन्होंने लद्दाख के खारदुंगला में तिरंगा फहराया।
मेरे प्यारे देशवासियों, गणेश उत्सव नजदीक आ रहा है।
स्वतंत्रता संग्राम में उनका अभूतपूर्व योगदान हमारे दिलों में जीवित 

In [6]:
# Open a file named "example.txt" in write mode ('w') with UTF-8 encoding
with open('B_eng_to_hin_output.txt', 'w', encoding='utf-8') as f:
    # Write some text to the file
    for i in hi_ans:
        f.writelines(i + '\n')


## Hindi to English

In [8]:
from google.colab import drive
import random
# Mount Google Drive
drive.mount('/content/drive')

# Access files in Google Drive
path = "/content/drive/My Drive/Colab Notebooks/B_hin_to_eng_input.txt"

with open(path, "r", encoding="utf-8") as file:
    hi_data= file.readlines()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
print(len(hi_data))
for i in hi_data:
  print(i)

1000
आधुनिक जीवन के तनाव से शांति की खोज में अक्‍सर लोग तंबाकू, शराब और यहां तक कि ड्रग्‍स आदि नशीले पदार्थों का सेवन करने लगते हैं।

इसके तहत 100 मिलियन से भी अधिक गरीब एवं कमजोर परिवारों को प्रति वर्ष 7,000 डॉलर से भी अधिक राशि का बीमा कवर मिलेगा।

मेरे प्यारे देशवासियो, हर बार की तरह इस बार भी मुझे आप सबके सारे पत्र,email, phone-call और comments बहुत बड़ी मात्रा में मिले हैं।

अनगिनत घटनाएँ होंगी।

• प्रधानमंत्री द्वारा आज घोषित की गई आर्थिक मदद, इस महीने की शुरूआत में तमिलनाडु को 280 करोड़ रुपये और केरल को दी गई 76 करोड़ रुपये की आर्थिक मदद से अलग है।

भारत में प्रतिभाशाली वकीलों और न्यायाधीशों की कोई कमी नहीं है।

ब्राजील लैटिन अमेरिका में हमारे सबसे महत्वपूर्ण आर्थिक भागीदारों में से एक है।

लोग इस पवित्र माह में ज़रूरतमंदों को दान देते हैं।

सार्वजनिक गणेश उत्सव परम्परागत श्रद्धा और उत्सव के साथ-साथ समाज-जागरण, सामूहिकता, लोगों में समरसता और समानता के भाव को आगे बढ़ाने का एक प्रभावी माध्यम बन गया था।

प्रधानमंत्री ने पटना में रिवर फ्रंट डेवलपमेंट के पहले चरण का उद्घाटन किया।

स्वतं

In [10]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)

hi_sents = hi_data
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_ans = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)

# print(len(hi_data))
for i in en_ans:
  print(i)

# flush the models to free the GPU memory
# del indic_en_tokenizer, indic_en_model

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In search of peace from the stresses of modern life, people often start consuming drugs such as tobacco, alcohol, and even drugs.
Under this, more than 100 million poor and vulnerable families will get insurance cover of more than $7,000 per year.
My dear countrymen, like every time, this time also I have received all your letters, emails, phone-calls and comments in a very large amount.
There will be countless events.
The financial assistance announced by the Prime Minister today is in addition to the Rs 280 crore given to Tamil Nadu and Rs 76 crore given to Kerala earlier this month.
There is no dearth of talented lawyers and judges in India.
Brazil is one of our most important economic partners in Latin America.
People donate to the needy during this holy month.
Public Ganesh Utsav had become an effective medium of carrying forward the traditional reverence and celebration as well as the spirit of social-awakening, collectivity, harmony and equality among the people.
The Prime Minis

In [11]:

with open('B_hin_to_eng_output.txt', 'w', encoding='utf-8') as f:
    # Write some text to the file
    for i in en_ans:
        f.writelines(i + '\n')


## Marathi to Hindi

In [12]:
from google.colab import drive
import random
# Mount Google Drive
drive.mount('/content/drive')

# Access files in Google Drive
path = "/content/drive/My Drive/Colab Notebooks/B_mar_to_hin_input.txt"

with open(path, "r", encoding="utf-8") as file:
    mar_data = file.readlines()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
print(len(mar_data))
for i in mar_data:
  print(i)

1000
काही तासातच आपण सर्वजण 2017 या नव्या वर्षाचे स्वागत करू.

केंद्रीय रस्ते निधी अंतर्गत बांधण्यात येणाऱ्या रस्त्याचे भूमीपुजनही पंतप्रधान बटण दाबून करणार आहेत.

फक्त इमारतींची छायाचित्रे नाहीत, फक्त नैसर्गिक सौदर्यांची छायाचित्रे नाहीत तर तिथल्या जनजीवनाविषयी काही विशेष गोष्टी तुम्ही लिहा.

या सर्वांचा उद्देश देशात व्यापक परिवर्तन घडवणे आणि नवीन भारताच्या निर्माणाचा मार्ग प्रशस्त करणे हा आहे.

अशाच प्रकारे देशामध्ये केंद्र आणि राज्यांच्या निवडणुका एकाचवेळी करण्याविषयी खूप मोठ्या प्रमाणावर सध्या चर्चा सुरू आहे.

हे केंद्र डॉ. आंबेडकरांची शिकवण आणि त्यांची देशाविषयीची दृष्टी आजच्या युवकांपर्यंत पोहोचवण्यासाठी महत्वाची भूमिका बजावेल, असा विश्वास पंतप्रधानांनी यावेळी बोलतांना व्यक्त केला.

जातीयवादाचे विष, सांप्रदायिकतेचे विष, देशाचे कधीही भले करू शकत नाही.

बिहारच्या प्रगतीसाठी त्यांनी घेतलेले परिश्रम कौतुकास्पद आहेत.

या यशस्वी मोहिमेबद्दल पंतप्रधानांनी या महिला अधिकाऱ्यांचे अभिनंदन केले.

ओखी चक्रीवादळाचा फटका असलेल्या नागरिकांच्या विम्याच्या दाव्याची रक्कम झटपट देण्याचा सल्ला विमा क

In [14]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

hi_sents = mar_data
src_lang, tgt_lang = "mar_Deva", "hin_Deva"
hindi_ans= batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

for i in hindi_ans:
  print(i)

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

कुछ ही घंटों में हम सभी नए साल 2017 का स्वागत करेंगे।
प्रधानमंत्री केंद्रीय सड़क कोष के तहत बनाई जाने वाली सड़क की भूमि पूजन भी बटन दबाकर करेंगे।
न केवल इमारतों की तस्वीरें, न केवल प्राकृतिक सुंदरता की तस्वीरें, बल्कि आप वहां के लोगों के जीवन के बारे में कुछ विशेष भी लिख सकते हैं।
इन सभी का उद्देश्य देश में व्यापक परिवर्तन लाना और नए भारत के निर्माण का मार्ग प्रशस्त करना है।
इसी तरह, देश में केंद्र और राज्यों के चुनाव एक साथ कराने के बारे में काफी चर्चा चल रही है।
इस अवसर पर बोलते हुए, प्रधानमंत्री ने विश्वास व्यक्त किया कि यह केंद्र डॉ. अंबेडकर की शिक्षाओं और देश के प्रति उनकी दृष्टि को आज के युवाओं तक पहुँचाने में महत्वपूर्ण भूमिका निभाएगा।
साम्प्रदायिकता का विष, साम्प्रदायिकता का विष, कभी भी देश के लिए अच्छा नहीं कर सकता है।
बिहार की प्रगति के लिए उन्होंने जो मेहनत की है, वह सराहनीय है।
प्रधानमंत्री ने इन महिला अधिकारियों को इस सफल अभियान के लिए बधाई दी।
बीमा कंपनियों को चक्रवात ओखी से प्रभावित नागरिकों के बीमा दावों की राशि जल्द से जल्द देने की सलाह दी गई है।
इसके अलावा, बी. एस. एफ

In [15]:
with open('B_mar_to_hin_output.txt', 'w', encoding='utf-8') as f:
    # Write some text to the file
    for i in hindi_ans:
        f.writelines(i + '\n')


## Hindi to Marathi

In [16]:
from google.colab import drive
import random
# Mount Google Drive
drive.mount('/content/drive')

# Access files in Google Drive
path = "/content/drive/My Drive/Colab Notebooks/B_hin_to_mar_input.txt"

with open(path, "r", encoding="utf-8") as file:
    hindi_data = file.readlines()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
print(len(hindi_data))
for i in hindi_data:
  print(i)

1000
उसने तो अपनी जिंदगी भर की कमाई सिर्फ आपके एक sign पर भरोसा करके निवेश किया था।

इतने सारे युवा पहली बार अपने घरों को छोड़कर, अपने जीवन को एक नयी दिशा देने निकल आते हैं।

अन्‍य व्‍यवस्‍थाओं का तेजी से बढ़ना उत्‍पादकों के हित में है।

मुझे उम्मीद है कि नया Course इस Profession में आने वाले नए लोगों की Financial Skills को और मजबूत करेगा।

यह बदलाव है।

अंतरराष्‍ट्रीय योग दिवस पर इतने सारे देशों का एक साथ आना योग के वास्‍तविक सार- एकजुटता – को दर्शाता है।

इसलिए एक परम मित्र और भारत के प्रखर समर्थक का स्वागत करना मेरे लिए सम्मान की बात है।

उन्होंने कहा कि कृषि को रूपांतरित करने के लिए किसानों एवं वैज्ञानिकों को एक साथ मिल कर काम करने की जरुरत है।

यह बढ़ते भूमि मूल्‍य कब्‍जे में लेने के लिए एक तंत्र प्रदान करता है।

मेरा मानना है कि पठन और ज्ञान का दायरा केवल काम संबंधी पहलुओं तक सीमित नहीं होना चाहिए।

प्रधानमंत्री मोदी ने यूरोपीय संघ में एकजुटता के महत्व को दोहराया और दुनिया में स्थिरता के बल के रूप में इसके महत्व को रेखांकित किया।

बेटी शिवांगी को बहुत-बहुत बधाई।

उन्होंने बड़ी चिं

In [18]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

hi_sents =  hindi_data
src_lang, tgt_lang = "hin_Deva", "mar_Deva"
mar_ans = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

for i in mar_ans:
  print(i)

त्याने फक्त तुमच्या एका चिन्हावर अवलंबून राहून आयुष्यभरची कमाई केली होती.
अनेक तरुण त्यांच्या आयुष्यासाठी एक नवीन दिशा देण्यासाठी पहिल्यांदाच त्यांचे घर सोडून बाहेर पडतात.
इतर व्यवस्थांचा वेगवान विकास हा उत्पादकांच्या हितासाठी आहे.
मला आशा आहे की नवीन अभ्यासक्रम या व्यवसायात येणाऱ्या नवीन लोकांच्या आर्थिक कौशल्यांना आणखी बळकट करेल.
हा बदल आहे.
आंतरराष्ट्रीय योग दिनानिमित्त अनेक देश एकत्र येणे हे योगाचे खरे सार-एकात्मता-प्रतिबिंबित करते.
त्यामुळे एक परिपूर्ण मित्र आणि भारताचे प्रखर समर्थक यांचे स्वागत करणे हा माझा सन्मान आहे.
कृषी क्षेत्राचे रूपांतर करण्यासाठी शेतकरी आणि शास्त्रज्ञांनी एकत्र काम करण्याची गरज आहे, असे ते म्हणाले.
यामुळे जमिनीची वाढती किंमत ताब्यात घेण्यासाठी एक यंत्रणा उपलब्ध होते.
माझा असा विश्वास आहे की वाचन आणि ज्ञानाची व्याप्ती केवळ कामकाजाच्या पैलूंपुरती मर्यादित नसावी.
पंतप्रधान मोदी यांनी ई. यू. मधील एकजुटीचे महत्त्व अधोरेखित केले आणि जगातील स्थिरतेचे बळ म्हणून त्याचे महत्त्व अधोरेखित केले.
मुलगी शिवांगीला खूप खूप अभिनंदन.
दहशतवाद आणि हिंसक दहशतवादाच्या वाढीचा आणि

In [19]:

with open('B_hin_to_mar_output.txt', 'w', encoding='utf-8') as f:
    # Write some text to the file
    for i in mar_ans:
        f.writelines(i + '\n')
